## Identifying and resolving conflicts in a batch of VI files

Andes Reinspection 
ELG
Data information can be found at https://data.desi.lbl.gov/desi/spectro/prospect/target-reinspection-andes/

/global/cfs/cdirs/desi/spectro/redux/andes


448 visual inspections of a total of 150 unique objects

32 conflicts

In [52]:
save_filename='truth_table_Andes_reinspection_ELG_67230_20200315_5'

In [53]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

import VI_utils as VI
sys.path.append("/global/homes/r/tlan/prospect/prospect/py")

from prospect import utils_specviewer,plotframes
#import desispec
# library location will change ..
sys.path.append("/global/u2/t/tlan/python/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [54]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
#petals = ['0','3','6','7']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [55]:
pd.set_option('display.max_rows', 10)

In [56]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/ELG/'

In [57]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(13,15):
    pattern = "desi*_"+str(67230)+"_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

print(vi_files)

desi*_67230_13_*.csv
desi*_67230_14_*.csv
['desi-vi_ELG_reinspection_67230_13_MAR.csv', 'desi-vi_ELG_reinspection_67230_13_CGS.csv', 'desi-vi_ELG_reinspection_67230_14_MAR.csv', 'desi-vi_ELG_reinspection_67230_14_CGS.csv']


In [58]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = ",", engine='python')
print(vi_files[0])
for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = ",", engine='python')
    vi = vi.append(vi2, ignore_index=True)
#print(vi)    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TARGETID'])

print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_ELG_reinspection_67230_13_MAR.csv
desi-vi_ELG_reinspection_67230_13_CGS.csv
desi-vi_ELG_reinspection_67230_14_MAR.csv
desi-vi_ELG_reinspection_67230_14_CGS.csv
There are 160 visual inspections of a total of 80 unique objects
That gives 2.0 inspections per object.


In [59]:
#vi is a dataframe
#display(vi)

In [60]:
#vi.keys()
len(vi)

160

# Merge with zbest files
#Add: fiberID, delta_chi2, flux information,.. anything else?


In [61]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_panda()    
EXPID = list(set(tf['EXPID']))[0]
tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()
print(len(tspec_df))
print(len(tf_df))
#tf_df = tf_df.rename(columns={"TargetID": "TARGETID"})
#tspec_df = tspec_df.rename(columns={"TargetID": "TARGETID"})
#print(len(vi))
vi = vi.merge(tf_df, how='left', on='TARGETID')
vi = vi.merge(tspec_df, how='left', on='TARGETID')
print(len(vi))

5000
5000
160


In [62]:
print(vi.keys())
#print(len(vi))
#vi['best redshift']=vi['VI_z']

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')


### Adding a bunch of useful columns

In [63]:
#vi['VI_z']
import VI_utils as VI
#vi['VI_z']
#bad_str = [s is None for s in vi['VI_comment']]
#vi['VI_comment'].isnull()
#bad_str = [s is None for s in vi['VI_comment']]
#print(bad_str)
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['VI_comment']
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))
#print(vi['all VI comments'])
vi = VI.prep_new_columns(vi)
#vi['best redshift'] = vi['VI_z']
#test = vi['VI_z']
#vi['best redshift'].isnull()
#vi.loc[vi['best redshift'].isnull(), 'best redshift'] = vi.loc[vi['best redshift'].isnull(), 'Redrock_z']
#vi['best redshift']

In [64]:
#check all the new columns (keys) have been added correctly
display(vi)
#print(vi.keys())

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185802342302085,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.8196,MAR,...,23.448356,1.4451,GALAXY,2.0,0,0.000368,1.44465,--|step/jump in the last channel,2,none
1,35185802342302092,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.8388,MAR,...,263.361098,0.8388,GALAXY,4.0,0,0.000000,0.83880,--|--,2,none
2,35185802342302740,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.7619,MAR,...,106.914634,0.7622,GALAXY,3.0,2,0.000170,0.76205,--|--,2,none
3,35185802342303829,55587,20200315,67230,0.34.4,-1,-1,GALAXY,1.2497,MAR,...,119.537663,1.2497,GALAXY,4.0,0,0.000000,1.24970,--|--,2,none
4,35185802342303898,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.1905,MAR,...,67.733652,0.1905,GALAXY,2.5,1,0.000000,0.19050,--|--,2,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,35185802371667169,55587,20200315,67230,0.34.4,-1,-1,QSO,2.3077,CGS,...,24.301832,2.3077,QSO,0.5,1,0.000000,2.30770,--|--,2,none
156,35185802375856417,55587,20200315,67230,0.34.4,-1,-1,GALAXY,1.6065,CGS,...,16.259349,1.6071,GALAXY,1.5,1,0.000230,1.60680,--|--,2,none
157,35185802375856933,55587,20200315,67230,0.34.4,-1,-1,GALAXY,1.5982,CGS,...,168.899621,1.5982,GALAXY,2.0,0,0.000038,1.59815,--|--,2,none
158,35185802375857132,55587,20200315,67230,0.34.4,-1,-1,GALAXY,0.7850,CGS,...,16.580985,0.7850,GALAXY,0.5,1,0.000000,0.78500,--|--,2,none


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [65]:
vi_gp = vi.groupby(['TARGETID'])
vi_conflict = VI.find_conflicts(vi_gp)

In [66]:
len(vi_gp)

80

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [67]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185796336062284 35185796340253548 35185796340255905 35185796344449049
 35185802342302740 35185802359080077 35185802363276778 35185802363277027
 35185802367473057 35185802371662064 35185802371663154 35185802371666288
 35185802371666973 35185802375857926]
Total number of conflicts to resolve:  14


In [68]:
unique_targets[0]

35185796336062284

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [69]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TARGETID==unique_targets[conflict_id]][['TARGETID', 
       'Redrock_spectype', 'Redrock_z', 'VI_scanner', 'VI_quality', 'VI_issue',
       'VI_z', 'VI_spectype', 'VI_comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [70]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [71]:
show_spectra = False

In [72]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
42,35185796336062284,GALAXY,1.3336,MAR,0,NaN,NaN,NaN,--,1.3336,GALAXY,1.5,3,1.3336,0.0,--|--,2,none
92,35185796336062284,GALAXY,1.3336,CGS,3,NaN,NaN,NaN,--,1.3336,GALAXY,1.5,3,1.3336,0.0,--|--,2,none


In [73]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0885
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Broad [OII] with continuum'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

42    3
92    3
Name: VI_quality, dtype: int64
conflict ID =  0


In [74]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
46,35185796340253548,GALAXY,1.6349,MAR,1,NaN,NaN,NaN,--,1.6349,GALAXY,2.5,3,1.6349,0.169559,--|--,2,none
96,35185796340253548,GALAXY,1.6349,CGS,4,RS,1.2529,NaN,--,1.2529,GALAXY,2.5,3,1.6349,0.169559,--|--,2,none


In [75]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.2529
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] =4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Quality disagreement. Redshift is secure. Bad calibration'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)



46    4
96    4
Name: VI_quality, dtype: int64
conflict ID =  1


In [76]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
44,35185796340255905,GALAXY,0.9413,MAR,0,NaN,NaN,NaN,--,0.9413,GALAXY,2.0,4,0.9413,0.343121,--|--,2,none
94,35185796340255905,GALAXY,0.9413,CGS,4,R,1.6074,NaN,--,1.6074,GALAXY,2.0,4,0.9413,0.343121,--|--,2,none


In [77]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.6074
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. [OII] is secure.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  2


In [78]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
49,35185796344449049,GALAXY,0.5039,MAR,0,NaN,NaN,NaN,--,0.5039,GALAXY,2.0,4,0.5039,0.293171,--|--,2,none
99,35185796344449049,GALAXY,0.5039,CGS,4,RS,0.9448,NaN,--,0.9448,GALAXY,2.0,4,0.5039,0.293171,--|--,2,none


In [79]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.9448
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. Redshift is secure. Bad calibration'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)


49    4
99    4
Name: VI_quality, dtype: int64
conflict ID =  3


In [80]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
2,35185802342302740,GALAXY,0.7619,MAR,2,NaN,0.7622,NaN,--,0.7622,GALAXY,3.0,2,0.76205,0.00017,--|--,2,none
52,35185802342302740,GALAXY,0.7619,CGS,4,NaN,NaN,NaN,--,0.7619,GALAXY,3.0,2,0.76205,0.00017,--|--,2,none


In [81]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.7622
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Redshift is secure'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

2     4
52    4
Name: VI_quality, dtype: int64
conflict ID =  4


In [82]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
37,35185802359080077,GALAXY,0.9747,MAR,3,NaN,1.0403,NaN,--,1.0403,GALAXY,1.5,3,0.9747,0.03322,--|--,2,none
87,35185802359080077,GALAXY,0.9747,CGS,0,NaN,NaN,NaN,--,0.9747,GALAXY,1.5,3,0.9747,0.03322,--|--,2,none


In [83]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in type.  Not obviou features'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  5


In [84]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
107,35185802363276778,GALAXY,1.0694,MAR,2,NaN,NaN,NaN,--,1.0694,GALAXY,3.0,2,1.0694,0.0,--|--,2,none
137,35185802363276778,GALAXY,1.0694,CGS,4,S,NaN,NaN,--,1.0694,GALAXY,3.0,2,1.0694,0.0,--|--,2,none


In [85]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.5
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is secure'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  6


In [86]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
108,35185802363277027,GALAXY,0.5654,MAR,2,NaN,1.0406,NaN,--,1.0406,GALAXY,2.5,1,0.5654,0.303565,--|--,2,none
138,35185802363277027,GALAXY,0.5654,CGS,3,S,NaN,NaN,--,0.5654,GALAXY,2.5,1,0.5654,0.303565,--|--,2,none


In [87]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1847
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Reshift is not secure. Bad calibration'

##look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  7


In [88]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
111,35185802367473057,GALAXY,0.9837,MAR,2,NaN,NaN,NaN,--,0.9837,GALAXY,3.0,2,0.9837,0.0,--|--,2,none
141,35185802367473057,GALAXY,0.9837,CGS,4,NaN,NaN,NaN,--,0.9837,GALAXY,3.0,2,0.9837,0.0,--|--,2,none


In [89]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1920
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = "Broad [OII]. Calibration features"

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  8


In [90]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
115,35185802371662064,GALAXY,1.4108,MAR,2,NaN,NaN,NaN,--,1.4108,GALAXY,3.0,2,1.4108,0.0,--|--,2,none
145,35185802371662064,GALAXY,1.4108,CGS,4,NaN,NaN,NaN,--,1.4108,GALAXY,3.0,2,1.4108,0.0,--|--,2,none


In [91]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1327
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. [OII] on sky lines. Should be robust.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  9


In [92]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
116,35185802371663154,GALAXY,1.0381,MAR,2,NaN,NaN,NaN,--,1.0381,GALAXY,3.0,2,1.03795,0.000147,--|--,2,none
146,35185802371663154,GALAXY,1.0381,CGS,4,R,1.0378,NaN,--,1.0378,GALAXY,3.0,2,1.03795,0.000147,--|--,2,none


In [93]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.6283
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Broad [OII]. flag disagreement.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  10


In [94]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
123,35185802371666288,GALAXY,1.1385,MAR,2,NaN,1.139,NaN,--,1.1390,GALAXY,3.0,2,1.13875,0.000234,--|--,2,none
153,35185802371666288,GALAXY,1.1385,CGS,4,NaN,NaN,NaN,--,1.1385,GALAXY,3.0,2,1.13875,0.000234,--|--,2,none


In [95]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0662
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Possibe weak OII. Be conservative. Could be 3'
#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


conflict ID =  11


In [96]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
124,35185802371666973,GALAXY,0.6206,MAR,2,NaN,1.177,NaN,--,1.1770,GALAXY,2.5,1,0.6206,0.34333,--|--,2,none
154,35185802371666973,GALAXY,0.6206,CGS,3,NaN,NaN,NaN,--,0.6206,GALAXY,2.5,1,0.6206,0.34333,--|--,2,none


In [97]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0810
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Broad [OII]. Not sure 3 or 2. Be conservative'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  12


In [98]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
129,35185802375857926,GALAXY,1.4156,MAR,2,NaN,NaN,NaN,--,1.4156,GALAXY,3.0,2,1.4156,0.064657,--|--,2,none
159,35185802375857926,GALAXY,1.4156,CGS,4,R,1.2689,NaN,--,1.2689,GALAXY,3.0,2,1.4156,0.064657,--|--,2,none


In [99]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Two emission line peaks can not explain.'

#look at the values again to make sure all is well
#display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



conflict ID =  13


### and so on...

We should now recompute the conflicts, and not find any.

In [100]:
vi_conflict = VI.find_conflicts(vi_gp)

In [101]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

Let's check the number of conflicts again.

In [102]:
vi_conflict = VI.find_conflicts(vi_gp)

In [103]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [104]:
vi['vi_combined_flag'] = vi.groupby('TARGETID')['VI_quality'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TARGETID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [246]:
vi.keys()

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2', 'best redshift',
       'best spectype', 'vi_combined_flag', 'vi_diff', 'dz', 'vi_combined_z',
       'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [105]:
print(VI_dir+save_filename)

/global/homes/t/tlan/projects/VI_files/Andes_reinspection/ELG/truth_table_Andes_reinspection_ELG_67230_20200315_5


In [106]:
vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',  
      'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER'].first().to_csv(VI_dir+save_filename)

<ipython-input-106-2063ed20beca>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',


Check that merged file reads in OK - check comments

In [107]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [108]:
merged_file.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'all VI comments',
       'merger comment', 'N_VI', 'DELTACHI2', 'FIBER'],
      dtype='object')

In [109]:
len(merged_file)

80

In [110]:
VI_dir

'/global/homes/t/tlan/projects/VI_files/Andes_reinspection/ELG/'